# Parking Spot Detection Notebook

This notebook demonstrates a workflow for detecting and classifying parking spots as either "Occupied" or "Empty" in a video feed. The process involves:

1. **Loading Required Libraries**: Importing necessary libraries for image processing, machine learning, and video manipulation.
2. **Defining Constants**: Setting paths for the model, video, and mask files, as well as image size for preprocessing.
3. **Helper Functions**: Implementing functions for extracting parking spot bounding boxes, preparing images for the model, and predicting the status of parking spots.
4. **Model Loading**: Loading a pre-trained TensorFlow model for classification.
5. **Video Processing**: Reading the video feed, applying the mask, and detecting parking spot statuses in real-time.


In [ ]:
# Data manipulation
import tensorflow as tf
import cv2
# from google.colab.patches import cv2_imshow # for displaying images in colab

In [ ]:
# Models directory
MODELS_DIRECTORY = '../model/model.keras'

# Video path
VIDEO_PATH = '../video/parking_1920_1080_loop.mp4'

# Mask path
MASK_PATH = '../video//mask_1920_1080.png'

In [3]:
IMG_SIZE = (224, 224)

In [ ]:
# Get the parking spots bounding boxes
def get_parking_spots_bboxes(connected_components):
    """
    Get the parking spots bounding boxes from the connected components.

    Parameters:
        -connected_components : tuple
    Returns
        - parking_spots_bboxes : list of tuples
    """
    # connected_components
    (totalLabels, label_ids, values, centroid) = connected_components

    # Get the parking spots bounding boxes
    parking_spots_bboxes = []

    # Get the parking spots bounding boxes
    # The first label is the background, so we start from 1
    coef = 1
    for i in range(1, totalLabels):
        x = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)
        parking_spots_bboxes.append((x, y, w, h))
        
    return parking_spots_bboxes


In [ ]:
# Preape the image for the model
def load_and_prepare_image(image):
    """
    Load and prepare the image for the model.

    Parameters:
        - image : numpy array

    Returns:
        - img : TensorFlow tensor
    """
    
    # Convert to Tensor and ensure it's float32
    img = tf.convert_to_tensor(image, dtype=tf.float32)

    # Ensure image has 3 channels
    if img.shape[-1] != 3:
        raise ValueError(f"Expected image with 3 channels, got shape: {img.shape}")

    # Resize to match model input
    img = tf.image.resize(img, IMG_SIZE)

    # Add batch dimension
    img = tf.expand_dims(img, axis=0)

    return img

In [6]:
# Load the model
model = tf.keras.models.load_model(MODELS_DIRECTORY)

c:\Users\Arturo\Documents\Annie_DAML_course\comp_vi_project\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Predict empty of not empty
def empty_or_not(spot_crop):
    """
    Predict if the parking spot is empty or not.

    Parameters:
        - spot_crop : numpy array

    Returns:
        - status : str
    """
    
    # Load and prepare image
    image = load_and_prepare_image(spot_crop)

    # Predict
    y_out = model.predict(image)

    if y_out > 0.5:
        status = 'Occupied'
    else:
        status = 'Empty'

    return status

In [8]:
# Load video and mask and connect
cap = cv2.VideoCapture(VIDEO_PATH)
mask = cv2.imread(MASK_PATH, cv2.IMREAD_GRAYSCALE)
connected_components = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)

# Get the parking spots
parking_spots_bboxes = get_parking_spots_bboxes(connected_components)

In [ ]:
ret = True
step = 30
spots_status = [None for _ in parking_spots_bboxes]
frame_nmr = 0

# Loop through the video frames
while ret:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_nmr % step == 0:
      
      for spot_id, parking_spot_bbox in enumerate(parking_spots_bboxes):

          x, y, w, h = parking_spot_bbox
          spot_crop = frame[y:y+h, x:x+w, :]

          spot_status = empty_or_not(spot_crop)
          spots_status[spot_id] = spot_status

    occupied_count = spots_status.count('Occupied')  # Count 'Occupied' in the list

    # Draw the parking spots on the frame
    for spot_id, parking_spot_bbox in enumerate(parking_spots_bboxes):

      spot_status = spots_status[spot_id]
      x, y, w, h = parking_spot_bbox

      if spot_status == 'Occupied':
          color = (0, 0, 255)
      else:
          color = (0, 255, 0)
      frame = cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, f"Available spots: {len(spots_status) - occupied_count} / {len(spots_status)}", (100, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow("Frame", frame) # Comment if not running in Google Colab
    #cv2_imshow(frame) # Uncomment if running in Google Colab
    frame_nmr += 1

    if frame_nmr > 120:
        print("Stopping after 120 frames")
        break

cap.release()
cv2.destroyAllWindows()